<a href="https://colab.research.google.com/github/NihPat95/Instacart-Market-Analysis/blob/master/Data_Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
from IPython.display import display

In [0]:
# Explore the orders csv file
print('orders.csv')
orders = pd.read_csv('orders.csv')
print('Columns \n{} \n'.format(orders.columns))
print('Shape \n{} \n'.format(orders.shape))
print('Info \n{} \n'.format(orders.info()))
orders.drop(columns=['eval_set', 'order_number', 'order_dow', 
                     'order_hour_of_day', 'days_since_prior_order'], 
            inplace=True)
display(orders.head())
print('-' * 100)

# Explore the order product prior and train files
print('order_products__prior.csv')
order_products__prior = pd.read_csv('order_products__prior.csv')
print('Columns \n{} \n'.format(order_products__prior.columns))
print('Shape \n{} \n'.format(order_products__prior.shape))
print('Info \n{} \n'.format(order_products__prior.info()))
order_products__prior.drop(columns=['order_id', 'product_id'], 
                           inplace=True)
display(order_products__prior.head())
print('-' * 100)

print('order_products__train.csv')
order_products__train = pd.read_csv('order_products__train.csv')
print('Columns \n{} \n'.format(order_products__train.columns))
print('Shape \n{} \n'.format(order_products__train.shape))
print('Info \n{} \n'.format(order_products__train.info()))
order_products__train.drop(columns=['order_id', 'product_id'], 
                           inplace=True)
display(order_products__train.head())

orders.csv
Columns 
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object') 

Shape 
(3421083, 7) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
Info 
None 



,order_id,user_id
0,2539329,1
1,2398795,1
2,473747,1
3,2254736,1
4,431534,1


----------------------------------------------------------------------------------------------------
order_products__prior.csv
Columns 
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object') 

Shape 
(32434489, 4) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB
Info 
None 



,add_to_cart_order,reordered
0,1,1
1,2,1
2,3,0
3,4,1
4,5,0


----------------------------------------------------------------------------------------------------
order_products__train.csv
Columns 
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object') 

Shape 
(1384617, 4) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             1384617 non-null int64
product_id           1384617 non-null int64
add_to_cart_order    1384617 non-null int64
reordered            1384617 non-null int64
dtypes: int64(4)
memory usage: 42.3 MB
Info 
None 



,add_to_cart_order,reordered
0,1,1
1,2,1
2,3,0
3,4,0
4,5,1


In [0]:
# Concatenate the train and prior to create a our order_products dataset
orders_products = pd.concat([order_products__train, order_products__prior])
display(orders_products.head())

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [0]:
# Merge order_products with orders to get user information
customer_orders = pd.merge(orders, orders_products, 
                           how='left', left_on='order_id', right_on='order_id')
customer_orders.drop(columns=['eval_set', 'order_dow', 'order_hour_of_day', 'order_number',
                              'days_since_prior_order', 'reordered', 'add_to_cart_order'], 
                     inplace=True)
display(customer_orders.head())

,order_id,user_id,product_id
0,2539329,1,196.0
1,2539329,1,14084.0
2,2539329,1,12427.0
3,2539329,1,26088.0
4,2539329,1,26405.0


In [0]:
orders_products.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [0]:
# Group by order id to create our basket
customer_orders['product_id'] = customer_orders['product_id'].astype(np.int32)
customer_orders = customer_orders.groupby(['order_id', 'user_id'])['product_id'].agg(lambda x: list(x))
customer_orders.to_csv('customer_orders.csv')
%time

ValueError: ignored

In [0]:
customer_orders.head()

order_id
1    [49302.0, 11109.0, 10246.0, 49683.0, 43633.0, ...
2    [33120.0, 28985.0, 9327.0, 45918.0, 30035.0, 1...
3    [33754.0, 24838.0, 17704.0, 21903.0, 17668.0, ...
4    [46842.0, 26434.0, 39758.0, 27761.0, 10054.0, ...
5    [13176.0, 15005.0, 47329.0, 27966.0, 23909.0, ...
Name: product_id, dtype: object